In [2]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [81]:
def calculateAngle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2((b[1]-a[1]), b[0]-a[0]) - np.arctan2((b[1]-c[1]), b[0]-c[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360.0 - angle

    return angle

In [ ]:
cap = cv2.VideoCapture(0)

counter = 0
stage = None

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        result = pose.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if result.pose_landmarks:
            try:
                landmarks = result.pose_landmarks.landmark
                
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                angle = calculateAngle(shoulder, elbow, wrist)

                cv2.putText(image, str(angle),
                           tuple(np.multiply(elbow,[640,480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (225,225,225), 2, cv2.LINE_AA)


                if angle > 130:
                    stage = "down"
                if angle < 55 and stage == "down":
                    stage = "up"
                    counter += 1
                    print(counter)
                
            except Exception as e:
                print(f"Error: {e}")

        cv2.rectangle(image, (0,0), (255,73), (245,117,16), -1)
        
        cv2.putText(image, 'Reps:', (15,12),
                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10,60),
                     cv2.FONT_HERSHEY_SIMPLEX, 1.5, (225,225,225), 2, cv2.LINE_AA)
        
        cv2.putText(image, 'Stage:', (90,12),
                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (90,60),
                     cv2.FONT_HERSHEY_SIMPLEX, 1.5, (225,225,225), 2, cv2.LINE_AA)
        
        mp_drawing.draw_landmarks(image, result.pose_landmarks,  mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(0,0,225), thickness=3, circle_radius=2), #for joints
                                  mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2), #for connections
                                 )
        cv2.imshow('Make sure your left hand is in the frame!', image)
        key = cv2.waitKey(10)
        if key == 27:
            break
cap.release()
cv2.destroyAllWindows()